In [1]:
from pre_processing import df_all, df_week_player,df_week_team, metrics_classes
import pandas as pd
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import seaborn as sns
import numpy as np
import re
import os
import datetime
import altair as alt

In [2]:
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
pd.options.display.max_rows = 4000
pd.options.display.max_seq_items = 2000

In [3]:
df_all["Date"].min()

Timestamp('2021-07-01 00:00:00')

In [4]:
df_week_player.head()

,Player,Position,Team Name,Year,Week,Year-Week,Duration,Total Distance(m),Total Player Load,Acc 2m/s2 Total Effort,Acc 3m/s2 Total Effort,Dec 2m/s2 Total Effort,Dec 3m/s2 Total Effort,High Intensity Distance(m),Sprint Distance(m),Maximum Velocity(m/s),IMA COD(left),IMA COD(right),Load Per Minute,Distance Per Minute,Acc-Dec-COD Per Minute,IMA COD Imbalance,Is IMA Imbalance,IMA COD Deviation,IMA COD(Right) %,IMA COD(Left) %,Duration EWMA ACWR,Total Distance(m) EWMA ACWR,Total Player Load EWMA ACWR,Acc 2m/s2 Total Effort EWMA ACWR,Acc 3m/s2 Total Effort EWMA ACWR,Dec 2m/s2 Total Effort EWMA ACWR,Dec 3m/s2 Total Effort EWMA ACWR,High Intensity Distance(m) EWMA ACWR,Sprint Distance(m) EWMA ACWR,Maximum Velocity(m/s) EWMA ACWR,IMA COD(left) EWMA ACWR,IMA COD(right) EWMA ACWR,Load Per Minute EWMA ACWR,Distance Per Minute EWMA ACWR,Acc-Dec-COD Per Minute EWMA ACWR,is_Duration_abnormal,is_Total Distance(m)_abnormal,is_Total Player Load_abnormal,is_Acc 2m/s2 Total Effort_abnormal,is_Acc 3m/s2 Total Effort_abnormal,is_Dec 2m/s2 Total Effort_abnormal,is_Dec 3m/s2 Total Effort_abnormal,is_High Intensity Distance(m)_abnormal,is_Sprint Distance(m)_abnormal,is_Maximum Velocity(m/s)_abnormal,is_IMA COD(left)_abnormal,is_IMA COD(right)_abnormal,is_Load Per Minute_abnormal,is_Distance Per Minute_abnormal,is_Acc-Dec-COD Per Minute_abnormal,Intensity Risk Score,Agility Risk Score,IMA Risk Score,Volumn Risk Score,Date,Season
0,Player1,Central Midfielder,Team1,2021,26,2021-W26,102.0,5162.00,491.00,3.0,0.0,7.0,2.0,96.70,0.00,25.70,50.0,39.0,4.81,50.61,0.97,0.28,True,Right,43.82%,56.18%,1.00,1.00,1.00,1.00,NaN,1.00,1.00,1.00,NaN,1.00,1.00,1.00,1.00,1.00,1.00,Moderate,Moderate,Moderate,Moderate,Moderate,Moderate,Moderate,Moderate,Moderate,Moderate,Moderate,Moderate,Moderate,Moderate,Moderate,0,0,0,0,2021-06-28,None
1,Player1,Central Midfielder,Team1,2021,27,2021-W27,51.2,5077.60,480.48,16.0,3.8,14.6,3.2,231.94,41.08,27.10,15.8,19.0,9.38,99.17,1.28,-0.17,True,Right,54.60%,45.40%,0.75,0.99,0.99,1.37,1.50,1.21,1.14,1.24,1.50,1.02,0.58,0.74,1.19,1.20,1.09,Low,Moderate,Moderate,Moderate,Moderate,Moderate,Moderate,Moderate,Moderate,Moderate,Low,Low,Moderate,Moderate,Moderate,0,0,2,1,2021-07-05,2021/22
2,Player1,Central Midfielder,Team1,2021,28,2021-W28,55.4,4672.20,501.36,16.0,2.6,16.2,4.8,202.18,12.28,27.40,24.8,27.6,9.05,84.34,1.53,-0.10,False,Right,52.67%,47.33%,0.91,0.96,1.02,1.13,1.01,1.12,1.22,1.03,0.65,1.01,0.96,1.03,1.06,1.01,1.11,Moderate,Moderate,Moderate,Moderate,Moderate,Moderate,Moderate,Moderate,Low,Moderate,Moderate,Moderate,Moderate,Moderate,Moderate,0,0,0,0,2021-07-12,2021/22
3,Player1,Central Midfielder,Team1,2021,29,2021-W29,57.6,4666.34,523.60,18.6,3.6,16.0,6.0,219.10,20.46,28.24,15.8,16.0,9.09,81.01,1.15,-0.01,False,Right,50.31%,49.69%,0.97,0.98,1.03,1.13,1.15,1.05,1.19,1.05,1.04,1.02,0.77,0.76,1.03,0.98,0.92,Moderate,Moderate,Moderate,Moderate,Moderate,Moderate,Moderate,Moderate,Moderate,Moderate,Low,Low,Moderate,Moderate,Moderate,0,0,2,0,2021-07-19,2021/22
4,Player1,Central Midfielder,Team1,2021,30,2021-W30,65.6,5769.14,561.52,16.8,1.4,16.8,7.0,233.08,14.08,27.44,22.4,25.4,8.56,87.94,1.24,-0.12,True,Right,53.14%,46.86%,1.05,1.09,1.05,1.01,0.63,1.05,1.16,1.05,0.84,1.00,1.04,1.09,0.98,1.03,0.99,Moderate,Moderate,Moderate,Moderate,Low,Moderate,Moderate,Moderate,Moderate,Moderate,Moderate,Moderate,Moderate,Moderate,Moderate,0,1,0,0,2021-07-26,2021/22


In [21]:
player1 = df_week_player[df_week_player["Player"]=="Player1"]
player1.head()

,Player,Position,Team Name,Year,Week,Year-Week,Duration,Total Distance(m),Total Player Load,Acc 2m/s2 Total Effort,Acc 3m/s2 Total Effort,Dec 2m/s2 Total Effort,Dec 3m/s2 Total Effort,High Intensity Distance(m),Sprint Distance(m),Maximum Velocity(m/s),IMA COD(left),IMA COD(right),Load Per Minute,Distance Per Minute,Acc-Dec-COD Per Minute,IMA COD Imbalance,Is IMA Imbalance,IMA COD Deviation,IMA COD(Right) %,IMA COD(Left) %,Duration EWMA ACWR,Total Distance(m) EWMA ACWR,Total Player Load EWMA ACWR,Acc 2m/s2 Total Effort EWMA ACWR,Acc 3m/s2 Total Effort EWMA ACWR,Dec 2m/s2 Total Effort EWMA ACWR,Dec 3m/s2 Total Effort EWMA ACWR,High Intensity Distance(m) EWMA ACWR,Sprint Distance(m) EWMA ACWR,Maximum Velocity(m/s) EWMA ACWR,IMA COD(left) EWMA ACWR,IMA COD(right) EWMA ACWR,Load Per Minute EWMA ACWR,Distance Per Minute EWMA ACWR,Acc-Dec-COD Per Minute EWMA ACWR,is_Duration_abnormal,is_Total Distance(m)_abnormal,is_Total Player Load_abnormal,is_Acc 2m/s2 Total Effort_abnormal,is_Acc 3m/s2 Total Effort_abnormal,is_Dec 2m/s2 Total Effort_abnormal,is_Dec 3m/s2 Total Effort_abnormal,is_High Intensity Distance(m)_abnormal,is_Sprint Distance(m)_abnormal,is_Maximum Velocity(m/s)_abnormal,is_IMA COD(left)_abnormal,is_IMA COD(right)_abnormal,is_Load Per Minute_abnormal,is_Distance Per Minute_abnormal,is_Acc-Dec-COD Per Minute_abnormal,Intensity Risk Score,Agility Risk Score,IMA Risk Score,Volumn Risk Score,Date,Season
0,Player1,Central Midfielder,Team1,2021,26,2021-W26,102.0,5162.00,491.00,3.0,0.0,7.0,2.0,96.70,0.00,25.70,50.0,39.0,4.81,50.61,0.97,0.28,True,Right,43.82%,56.18%,1.00,1.00,1.00,1.00,NaN,1.00,1.00,1.00,NaN,1.00,1.00,1.00,1.00,1.00,1.00,Moderate,Moderate,Moderate,Moderate,Moderate,Moderate,Moderate,Moderate,Moderate,Moderate,Moderate,Moderate,Moderate,Moderate,Moderate,0,0,0,0,2021-06-28,None
1,Player1,Central Midfielder,Team1,2021,27,2021-W27,51.2,5077.60,480.48,16.0,3.8,14.6,3.2,231.94,41.08,27.10,15.8,19.0,9.38,99.17,1.28,-0.17,True,Right,54.60%,45.40%,0.75,0.99,0.99,1.37,1.50,1.21,1.14,1.24,1.50,1.02,0.58,0.74,1.19,1.20,1.09,Low,Moderate,Moderate,Moderate,Moderate,Moderate,Moderate,Moderate,Moderate,Moderate,Low,Low,Moderate,Moderate,Moderate,0,0,2,1,2021-07-05,2021/22
2,Player1,Central Midfielder,Team1,2021,28,2021-W28,55.4,4672.20,501.36,16.0,2.6,16.2,4.8,202.18,12.28,27.40,24.8,27.6,9.05,84.34,1.53,-0.10,False,Right,52.67%,47.33%,0.91,0.96,1.02,1.13,1.01,1.12,1.22,1.03,0.65,1.01,0.96,1.03,1.06,1.01,1.11,Moderate,Moderate,Moderate,Moderate,Moderate,Moderate,Moderate,Moderate,Low,Moderate,Moderate,Moderate,Moderate,Moderate,Moderate,0,0,0,0,2021-07-12,2021/22
3,Player1,Central Midfielder,Team1,2021,29,2021-W29,57.6,4666.34,523.60,18.6,3.6,16.0,6.0,219.10,20.46,28.24,15.8,16.0,9.09,81.01,1.15,-0.01,False,Right,50.31%,49.69%,0.97,0.98,1.03,1.13,1.15,1.05,1.19,1.05,1.04,1.02,0.77,0.76,1.03,0.98,0.92,Moderate,Moderate,Moderate,Moderate,Moderate,Moderate,Moderate,Moderate,Moderate,Moderate,Low,Low,Moderate,Moderate,Moderate,0,0,2,0,2021-07-19,2021/22
4,Player1,Central Midfielder,Team1,2021,30,2021-W30,65.6,5769.14,561.52,16.8,1.4,16.8,7.0,233.08,14.08,27.44,22.4,25.4,8.56,87.94,1.24,-0.12,True,Right,53.14%,46.86%,1.05,1.09,1.05,1.01,0.63,1.05,1.16,1.05,0.84,1.00,1.04,1.09,0.98,1.03,0.99,Moderate,Moderate,Moderate,Moderate,Low,Moderate,Moderate,Moderate,Moderate,Moderate,Moderate,Moderate,Moderate,Moderate,Moderate,0,1,0,0,2021-07-26,2021/22


In [6]:
not_pass_metrics = {}
for key, value in metrics_classes.items():
    not_pass = [metric for metric in value if any(player1[f"is_{metric}_abnormal"]!="Moderate")]
    not_pass_metrics[key] = not_pass

In [7]:
not_pass_metrics

{'Intensity': ['Acc-Dec-COD Per Minute'],
 'Agility': ['Acc 2m/s2 Total Effort',
  'Acc 3m/s2 Total Effort',
  'Dec 2m/s2 Total Effort',
  'Dec 3m/s2 Total Effort'],
 'IMA': ['IMA COD(left)', 'IMA COD(right)'],
 'Volumn': ['Total Distance(m)', 'Duration']}

In [8]:
plot_df = df_week_player[df_week_player["Player"] == "Player1"].dropna()

In [9]:
ewma_acwr = alt.Chart(plot_df).mark_line(point=alt.OverlayMarkDef(color="#FF7676"), color="#FA8072").encode(
alt.X("Date"),
alt.Y(f'Total Distance(m) EWMA ACWR'),
alt.Tooltip(["Date:T", "Total Distance(m) EWMA ACWR", "Total Distance(m):Q"])
)

# Create the dashed horizontal line at y = 1.5
upper_ewma_acwr = alt.Chart(pd.DataFrame({'y': [1.5]})).mark_rule(strokeDash=[5, 5], color='#FF6969').encode(
    y='y:Q'
)

# Create the dashed horizontal line at y = 0.8
lower_ewma_acwr = alt.Chart(pd.DataFrame({'y': [0.8]})).mark_rule(strokeDash=[5, 5], color='#FF6969').encode(
    y='y:Q'
)
    
chart = alt.Chart(plot_df).mark_bar(color="#0F52BA").encode(
    alt.X("Date:T", axis=alt.Axis(title="Date")),
    alt.Y("Total Distance(m):Q", axis=alt.Axis(title="Total Distance(m)")),
    alt.Tooltip(["Date:T", "Total Distance(m):Q", "Total Distance(m) EWMA ACWR"])
)

ewma_acwr_combined = alt.layer(
    ewma_acwr,
    upper_ewma_acwr,
    lower_ewma_acwr
).resolve_scale(
    y='shared'  # Share y-axis for the EWMA ACWR related charts
)


combined_chart = alt.layer(
    chart,
    ewma_acwr_combined
).resolve_scale(
    y='independent'  # Allow independent y-axes
).properties(
    title="Total Distance and EWMA ACWR Over Time"
)

In [10]:
combined_chart

alt.LayerChart(...)

In [11]:
metrics_classes

{'Intensity': ['Load Per Minute',
  'Distance Per Minute',
  'Acc-Dec-COD Per Minute'],
 'Agility': ['Acc 2m/s2 Total Effort',
  'Acc 3m/s2 Total Effort',
  'Dec 2m/s2 Total Effort',
  'Dec 3m/s2 Total Effort'],
 'IMA': ['IMA COD(left)', 'IMA COD(right)'],
 'Volumn': ['Total Distance(m)', 'Total Player Load', 'Duration']}

# balance

In [12]:
player1.head()

,Player,Position,Team Name,Year,Week,Year-Week,Duration,Total Distance(m),Total Player Load,Acc 2m/s2 Total Effort,Acc 3m/s2 Total Effort,Dec 2m/s2 Total Effort,Dec 3m/s2 Total Effort,High Intensity Distance(m),Sprint Distance(m),Maximum Velocity(m/s),IMA COD(left),IMA COD(right),Load Per Minute,Distance Per Minute,Acc-Dec-COD Per Minute,IMA COD Imbalance,Is IMA Imbalance,IMA COD Deviation,IMA COD(Right) %,IMA COD(Left) %,Duration EWMA ACWR,Total Distance(m) EWMA ACWR,Total Player Load EWMA ACWR,Acc 2m/s2 Total Effort EWMA ACWR,Acc 3m/s2 Total Effort EWMA ACWR,Dec 2m/s2 Total Effort EWMA ACWR,Dec 3m/s2 Total Effort EWMA ACWR,High Intensity Distance(m) EWMA ACWR,Sprint Distance(m) EWMA ACWR,Maximum Velocity(m/s) EWMA ACWR,IMA COD(left) EWMA ACWR,IMA COD(right) EWMA ACWR,Load Per Minute EWMA ACWR,Distance Per Minute EWMA ACWR,Acc-Dec-COD Per Minute EWMA ACWR,is_Duration_abnormal,is_Total Distance(m)_abnormal,is_Total Player Load_abnormal,is_Acc 2m/s2 Total Effort_abnormal,is_Acc 3m/s2 Total Effort_abnormal,is_Dec 2m/s2 Total Effort_abnormal,is_Dec 3m/s2 Total Effort_abnormal,is_High Intensity Distance(m)_abnormal,is_Sprint Distance(m)_abnormal,is_Maximum Velocity(m/s)_abnormal,is_IMA COD(left)_abnormal,is_IMA COD(right)_abnormal,is_Load Per Minute_abnormal,is_Distance Per Minute_abnormal,is_Acc-Dec-COD Per Minute_abnormal,Intensity Risk Score,Agility Risk Score,IMA Risk Score,Volumn Risk Score,Date,Season
0,Player1,Central Midfielder,Team1,2021,26,2021-W26,102.0,5162.00,491.00,3.0,0.0,7.0,2.0,96.70,0.00,25.70,50.0,39.0,4.81,50.61,0.97,0.28,True,Right,43.82%,56.18%,1.00,1.00,1.00,1.00,NaN,1.00,1.00,1.00,NaN,1.00,1.00,1.00,1.00,1.00,1.00,Moderate,Moderate,Moderate,Moderate,Moderate,Moderate,Moderate,Moderate,Moderate,Moderate,Moderate,Moderate,Moderate,Moderate,Moderate,0,0,0,0,2021-06-28,None
1,Player1,Central Midfielder,Team1,2021,27,2021-W27,51.2,5077.60,480.48,16.0,3.8,14.6,3.2,231.94,41.08,27.10,15.8,19.0,9.38,99.17,1.28,-0.17,True,Right,54.60%,45.40%,0.75,0.99,0.99,1.37,1.50,1.21,1.14,1.24,1.50,1.02,0.58,0.74,1.19,1.20,1.09,Low,Moderate,Moderate,Moderate,Moderate,Moderate,Moderate,Moderate,Moderate,Moderate,Low,Low,Moderate,Moderate,Moderate,0,0,2,1,2021-07-05,2021/22
2,Player1,Central Midfielder,Team1,2021,28,2021-W28,55.4,4672.20,501.36,16.0,2.6,16.2,4.8,202.18,12.28,27.40,24.8,27.6,9.05,84.34,1.53,-0.10,False,Right,52.67%,47.33%,0.91,0.96,1.02,1.13,1.01,1.12,1.22,1.03,0.65,1.01,0.96,1.03,1.06,1.01,1.11,Moderate,Moderate,Moderate,Moderate,Moderate,Moderate,Moderate,Moderate,Low,Moderate,Moderate,Moderate,Moderate,Moderate,Moderate,0,0,0,0,2021-07-12,2021/22
3,Player1,Central Midfielder,Team1,2021,29,2021-W29,57.6,4666.34,523.60,18.6,3.6,16.0,6.0,219.10,20.46,28.24,15.8,16.0,9.09,81.01,1.15,-0.01,False,Right,50.31%,49.69%,0.97,0.98,1.03,1.13,1.15,1.05,1.19,1.05,1.04,1.02,0.77,0.76,1.03,0.98,0.92,Moderate,Moderate,Moderate,Moderate,Moderate,Moderate,Moderate,Moderate,Moderate,Moderate,Low,Low,Moderate,Moderate,Moderate,0,0,2,0,2021-07-19,2021/22
4,Player1,Central Midfielder,Team1,2021,30,2021-W30,65.6,5769.14,561.52,16.8,1.4,16.8,7.0,233.08,14.08,27.44,22.4,25.4,8.56,87.94,1.24,-0.12,True,Right,53.14%,46.86%,1.05,1.09,1.05,1.01,0.63,1.05,1.16,1.05,0.84,1.00,1.04,1.09,0.98,1.03,0.99,Moderate,Moderate,Moderate,Moderate,Low,Moderate,Moderate,Moderate,Moderate,Moderate,Moderate,Moderate,Moderate,Moderate,Moderate,0,1,0,0,2021-07-26,2021/22


In [14]:
player1[["Player","Date","IMA COD(left)", "IMA COD(right)","IMA COD(Right) %", "IMA COD(Left) %", "IMA COD Imbalance"]].head()


,Player,Date,IMA COD(left),IMA COD(right),IMA COD(Right) %,IMA COD(Left) %,IMA COD Imbalance
0,Player1,2021-06-28,50.0,39.0,43.82%,56.18%,0.28
1,Player1,2021-07-05,15.8,19.0,54.60%,45.40%,-0.17
2,Player1,2021-07-12,24.8,27.6,52.67%,47.33%,-0.10
3,Player1,2021-07-19,15.8,16.0,50.31%,49.69%,-0.01
4,Player1,2021-07-26,22.4,25.4,53.14%,46.86%,-0.12


In [29]:
df_melted["Date"].max()

Timestamp('2022-08-29 00:00:00')

In [30]:
player1["Date"].max()

Timestamp('2023-12-25 00:00:00')

In [33]:
player1.dropna().tail()

,Player,Position,Team Name,Year,Week,Year-Week,Duration,Total Distance(m),Total Player Load,Acc 2m/s2 Total Effort,Acc 3m/s2 Total Effort,Dec 2m/s2 Total Effort,Dec 3m/s2 Total Effort,High Intensity Distance(m),Sprint Distance(m),Maximum Velocity(m/s),IMA COD(left),IMA COD(right),Load Per Minute,Distance Per Minute,Acc-Dec-COD Per Minute,IMA COD Imbalance,Is IMA Imbalance,IMA COD Deviation,IMA COD(Right) %,IMA COD(Left) %,Duration EWMA ACWR,Total Distance(m) EWMA ACWR,Total Player Load EWMA ACWR,Acc 2m/s2 Total Effort EWMA ACWR,Acc 3m/s2 Total Effort EWMA ACWR,Dec 2m/s2 Total Effort EWMA ACWR,Dec 3m/s2 Total Effort EWMA ACWR,High Intensity Distance(m) EWMA ACWR,Sprint Distance(m) EWMA ACWR,Maximum Velocity(m/s) EWMA ACWR,IMA COD(left) EWMA ACWR,IMA COD(right) EWMA ACWR,Load Per Minute EWMA ACWR,Distance Per Minute EWMA ACWR,Acc-Dec-COD Per Minute EWMA ACWR,is_Duration_abnormal,is_Total Distance(m)_abnormal,is_Total Player Load_abnormal,is_Acc 2m/s2 Total Effort_abnormal,is_Acc 3m/s2 Total Effort_abnormal,is_Dec 2m/s2 Total Effort_abnormal,is_Dec 3m/s2 Total Effort_abnormal,is_High Intensity Distance(m)_abnormal,is_Sprint Distance(m)_abnormal,is_Maximum Velocity(m/s)_abnormal,is_IMA COD(left)_abnormal,is_IMA COD(right)_abnormal,is_Load Per Minute_abnormal,is_Distance Per Minute_abnormal,is_Acc-Dec-COD Per Minute_abnormal,Intensity Risk Score,Agility Risk Score,IMA Risk Score,Volumn Risk Score,Date,Season
57,Player1,Central Midfielder,Team1,2022,31,2022-W31,78.200000,5752.620000,626.360000,17.000000,3.2,13.400000,3.400000,204.96,9.960000,29.10,27.600000,31.200000,8.01,73.56,1.14,-0.12,True,Right,53.06%,46.94%,0.99,0.93,0.95,1.11,1.06,0.95,0.92,0.86,0.48,1.00,1.11,1.00,0.96,0.94,1.05,Moderate,Moderate,Moderate,Moderate,Moderate,Moderate,Moderate,Moderate,Low,Moderate,Moderate,Moderate,Moderate,Moderate,Moderate,0,0,0,0,2022-08-01,2022/23
58,Player1,Central Midfielder,Team1,2022,32,2022-W32,76.000000,5830.060000,647.400000,13.200000,1.6,11.000000,3.000000,228.28,29.380000,27.32,21.400000,30.000000,8.52,76.71,0.99,-0.29,True,Right,58.37%,41.63%,0.98,0.97,0.99,0.93,0.69,0.88,0.89,0.98,1.18,0.97,0.92,0.98,1.01,0.99,0.96,Moderate,Moderate,Moderate,Moderate,Low,Moderate,Moderate,Moderate,Moderate,Moderate,Moderate,Moderate,Moderate,Moderate,Moderate,0,1,0,0,2022-08-08,2022/23
59,Player1,Central Midfielder,Team1,2022,33,2022-W33,80.333333,7667.166667,788.266667,14.333333,1.0,15.333333,6.666667,311.10,15.233333,28.60,21.333333,27.666667,9.81,95.44,0.98,-0.23,True,Right,56.46%,43.54%,1.02,1.12,1.09,1.00,0.60,1.10,1.33,1.14,0.76,1.01,0.96,0.95,1.07,1.10,0.97,Moderate,Moderate,Moderate,Moderate,Low,Moderate,Moderate,Moderate,Low,Moderate,Moderate,Moderate,Moderate,Moderate,Moderate,0,1,0,0,2022-08-15,2022/23
60,Player1,Central Midfielder,Team1,2022,34,2022-W34,74.600000,5139.780000,563.360000,22.000000,7.2,15.800000,5.200000,340.32,41.400000,28.96,28.000000,30.800000,7.55,68.90,1.29,-0.09,False,Right,52.38%,47.62%,0.97,0.86,0.88,1.21,1.63,1.06,1.02,1.11,1.35,1.01,1.11,1.03,0.91,0.89,1.12,Moderate,Moderate,Moderate,Moderate,High,Moderate,Moderate,Moderate,Moderate,Moderate,Moderate,Moderate,Moderate,Moderate,Moderate,0,1,0,0,2022-08-22,2022/23
61,Player1,Central Midfielder,Team1,2022,35,2022-W35,66.000000,3895.650000,465.000000,13.500000,2.5,11.500000,0.500000,66.75,0.000000,25.65,28.500000,28.500000,7.05,59.02,1.24,0.00,False,Right,50.00%,50.00%,0.92,0.79,0.84,0.85,0.72,0.87,0.18,0.36,0.00,0.94,1.06,0.97,0.92,0.86,1.04,Moderate,Low,Moderate,Moderate,Low,Moderate,Low,Low,Low,Moderate,Moderate,Moderate,Moderate,Moderate,Moderate,0,2,0,1,2022-08-29,2022/23


In [37]:
player1 = player1.dropna()

# Melt the DataFrame to long format for Altair
df_melted = player1.melt(id_vars=['Player', 'Date', 'IMA COD(Right) %', 'IMA COD(Left) %', 'IMA COD Imbalance'], 
                    value_vars=['IMA COD(left)', 'IMA COD(right)'], 
                    var_name='Type', 
                    value_name='Value').dropna()

# Define colors
colors = {'IMA COD(left)': '#0F52BA', 'IMA COD(right)': '#4A8CC7'}

# Create the stack bar chart
bar_chart = alt.Chart(df_melted).mark_bar().encode(
    x=alt.X('Date:T', axis=alt.Axis(title='Date')),
    y=alt.Y('sum(Value):Q', axis=alt.Axis(title='IMA COD')),
    color=alt.Color('Type:N', scale=alt.Scale(domain=['IMA COD(left)', 'IMA COD(right)'], range=[colors['IMA COD(left)'], colors['IMA COD(right)']])),
    tooltip=[
        alt.Tooltip('Date:T', title='Date'),
        alt.Tooltip('IMA COD(Left) %:N', title='IMA COD(Left) %'),
        alt.Tooltip('IMA COD(Right) %:N', title='IMA COD(Right) %')
    ]
).properties(
    width=600,
    height=400
)

# Create the line chart for IMA COD Imbalance
line_chart = alt.Chart(player1).mark_line(color='#FF7F3E').encode(
    x='Date:T',
    y=alt.Y('IMA COD Imbalance:Q', axis=alt.Axis(title='IMA COD Imbalance', titleColor='#FF7F3E')),
    tooltip=[
        alt.Tooltip('Date:T', title='Date'),
        alt.Tooltip('IMA COD Imbalance:Q', title='IMA COD Imbalance')
    ]
)

# Create the dashed horizontal line at y = 0.8
balance_line = alt.Chart(pd.DataFrame({'y': [0.0]})).mark_rule(strokeDash=[5, 5], color='#FF7F3E').encode(
    y='y:Q'
)

balance_combined = alt.layer(
    line_chart,
    balance_line
).resolve_scale(
    y='shared'  # Share y-axis for IMA COD balance
)


# Combine both charts
combined_chart = alt.layer(
    bar_chart,
    balance_combined
).resolve_scale(
    y='independent'  # Allow independent y-axes
).properties(
    title='IMA COD (Left & Right) and Imbalance'
)

combined_chart

alt.LayerChart(...)